### Imports

In [20]:
# IMPORTS
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
import tensorflow as tf
from tensorflow import keras
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


In [21]:
# Loading Full Pandas Datafrme
db = pd.read_csv('fifa.csv')
db.head()

,id,fifa_year,name,first_name,last_name,ws_team_name,ws_tournament_name,ws_tournament_id,ws_player_id,ws_season,age,height,weight,position,appearances,subs_on,minutes_played,goals,assists_total,yellow_cards,red_cards,shots_per_game,aerials_won_per_game,man_of_match,pass_success,tackles_per_game,interceptions_per_game,fouls_per_game,offsides_won_per_game,clearance_per_game,was_dribbled_per_game,outfielder_blocked_per_game,goal_own,key_pass_per_game,dribbles_won_per_game,fouls_given_per_game,offsides_given_per_game,dispossessed_per_game,turnovers_per_game,total_passes_per_game,accurate_crosses_per_game,accurate_long_passes_per_game,accurate_through_ball_per_game,fifa_pace,fifa_acceleration,fifa_sprint_speed,fifa_shooting,fifa_positioning,fifa_finishing,fifa_shot_power,fifa_long_shots,fifa_volleys,fifa_penalties,fifa_passing,fifa_vision,fifa_crossing,fifa_free_kick,fifa_short_passing,fifa_long_passing,fifa_curve,fifa_dribbling,fifa_agility,fifa_balance,fifa_reactions,fifa_ball_control,fifa_dribbling_min,fifa_composure,fifa_defense,fifa_marking,fifa_interceptions,fifa_heading_accuracy,fifa_standing_tackle,fifa_sliding_tackle,fifa_physical,fifa_jumping,fifa_stamina,fifa_strength,fifa_aggression,fifa_overall_score,fifa_overall_category
0,1,2021,Robert Lewandowski,Robert,Lewandowski,Bayern Munich,Bundesliga,12,29400,2019/2020,32,184,80,ST,41,0,3649,49,9,5.0,0.0,4.825806,2.416129,13,75.885572,0.790323,0.212903,1.066129,0.016129,0.609677,0.593548,0.016129,0,1.630645,1.535484,1.493548,0.474194,1.470968,2.056452,24.770968,0.261290,0.491935,0.114516,78,77,78,91,94,94,89,85,89,88,78,79,71,85,84,70,79,86,77,82,93,88,85,88,43,35,49,85,42,19,82,84,76,86,81,91,gold
1,2,2021,Lionel Messi,Lionel,Messi,Barcelona,LaLiga,12,11119,2019/2020,33,170,72,RW,41,2,3542,28,24,6.0,0.0,4.346591,0.339015,26,82.224473,0.443182,0.231061,0.678030,0.000000,0.015152,0.585227,0.000000,0,2.410985,6.320076,2.248106,0.259470,2.369318,1.857955,53.361742,0.596591,2.200758,0.568182,85,91,80,92,93,95,86,94,88,75,91,95,85,94,91,91,93,95,91,95,94,96,96,96,38,32,40,70,35,24,65,68,72,69,44,93,gold
2,3,2021,Neymar,Neymar da Silva,Santos Júnior,Paris Saint-Germain,Ligue 1,12,50835,2019/2020,28,175,68,LW,22,1,1907,16,10,5.0,1.0,4.080952,0.176190,10,77.566742,0.900000,0.276190,1.109524,0.000000,0.000000,1.028571,0.000000,0,2.442857,6.071429,4.280952,0.247619,3.104762,4.576190,52.485714,0.685714,1.790476,0.647619,91,94,89,85,87,87,80,84,87,92,86,90,85,89,87,81,88,94,96,83,91,95,95,93,36,35,36,62,30,29,59,62,81,50,51,91,gold
3,4,2021,Harry Kane,Harry,Kane,Tottenham,Premier League,12,83532,2019/2020,27,188,89,ST,34,0,3040,24,2,5.0,0.0,3.696552,2.075862,6,71.118574,0.610345,0.103448,1.037931,0.000000,0.917241,0.424138,0.120690,0,0.865517,1.282759,1.458621,0.644828,0.951724,1.872414,19.334483,0.168966,0.996552,0.186207,68,66,69,91,93,94,91,86,85,90,80,83,75,68,81,83,80,81,69,73,90,84,80,91,47,56,35,85,36,38,83,79,84,84,81,88,gold
4,5,2021,Toni Kroos,Toni,Kroos,Real Madrid,LaLiga,12,31772,2019/2020,30,183,76,CM,41,4,3196,5,6,3.0,0.0,1.980952,0.254762,3,93.646575,2.050000,1.078571,0.816667,0.014286,0.352381,1.742857,0.000000,0,2.871429,1.011905,1.364286,0.111905,0.602381,0.895238,75.038095,1.402381,7.804762,0.097619,54,57,52,81,76,76,89,86,82,73,91,90,88,84,93,93,86,81,63,71,89,89,80,88,71,72,79,58,73,60,69,32,75,73,60,88,gold


# Getting Overall Card Ranks by Category

## Data Preparation

In [17]:
# Feature Set for Predicting Card Level / Color
card_level_db = db[db.appearances >= 3]
card_level_db = pd.concat([card_level_db.iloc[:, 7:40].drop(columns=['ws_season']), card_level_db.iloc[:, -1]], axis=1)
card_level_db.head()
#rank_db = card_level_db.drop(columns=['fifa_overall_category'])
rank_db = pd.get_dummies(card_level_db, drop_first=True, columns=[ 'position'])
rank_db.drop(['appearances'], axis=1)
display(rank_db.head())

,ws_tournament_id,ws_player_id,age,height,weight,appearances,subs_on,minutes_played,goals,assists_total,yellow_cards,red_cards,shots_per_game,aerials_won_per_game,man_of_match,pass_success,tackles_per_game,interceptions_per_game,fouls_per_game,offsides_won_per_game,clearance_per_game,was_dribbled_per_game,outfielder_blocked_per_game,goal_own,key_pass_per_game,dribbles_won_per_game,fouls_given_per_game,offsides_given_per_game,dispossessed_per_game,turnovers_per_game,total_passes_per_game,fifa_overall_category,position_CB,position_CDM,position_CF,position_CM,position_GK,position_LB,position_LM,position_LW,position_LWB,position_RB,position_RM,position_RW,position_RWB,position_ST
0,12,29400,32,184,80,41,0,3649,49,9,5.0,0.0,4.825806,2.416129,13,75.885572,0.790323,0.212903,1.066129,0.016129,0.609677,0.593548,0.016129,0,1.630645,1.535484,1.493548,0.474194,1.470968,2.056452,24.770968,gold,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,12,11119,33,170,72,41,2,3542,28,24,6.0,0.0,4.346591,0.339015,26,82.224473,0.443182,0.231061,0.678030,0.000000,0.015152,0.585227,0.000000,0,2.410985,6.320076,2.248106,0.259470,2.369318,1.857955,53.361742,gold,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,12,50835,28,175,68,22,1,1907,16,10,5.0,1.0,4.080952,0.176190,10,77.566742,0.900000,0.276190,1.109524,0.000000,0.000000,1.028571,0.000000,0,2.442857,6.071429,4.280952,0.247619,3.104762,4.576190,52.485714,gold,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,12,83532,27,188,89,34,0,3040,24,2,5.0,0.0,3.696552,2.075862,6,71.118574,0.610345,0.103448,1.037931,0.000000,0.917241,0.424138,0.120690,0,0.865517,1.282759,1.458621,0.644828,0.951724,1.872414,19.334483,gold,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,12,31772,30,183,76,41,4,3196,5,6,3.0,0.0,1.980952,0.254762,3,93.646575,2.050000,1.078571,0.816667,0.014286,0.352381,1.742857,0.000000,0,2.871429,1.011905,1.364286,0.111905,0.602381,0.895238,75.038095,gold,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [18]:
# Change card rankings to numerical values (1,2,3)
rank_db['fifa_overall_category'] = pd.Categorical(rank_db['fifa_overall_category'])
rank_db['fifa_overall_category'] = rank_db.fifa_overall_category.cat.codes


rank_db.head()

,ws_tournament_id,ws_player_id,age,height,weight,appearances,subs_on,minutes_played,goals,assists_total,yellow_cards,red_cards,shots_per_game,aerials_won_per_game,man_of_match,pass_success,tackles_per_game,interceptions_per_game,fouls_per_game,offsides_won_per_game,clearance_per_game,was_dribbled_per_game,outfielder_blocked_per_game,goal_own,key_pass_per_game,dribbles_won_per_game,fouls_given_per_game,offsides_given_per_game,dispossessed_per_game,turnovers_per_game,total_passes_per_game,fifa_overall_category,position_CB,position_CDM,position_CF,position_CM,position_GK,position_LB,position_LM,position_LW,position_LWB,position_RB,position_RM,position_RW,position_RWB,position_ST
0,12,29400,32,184,80,41,0,3649,49,9,5.0,0.0,4.825806,2.416129,13,75.885572,0.790323,0.212903,1.066129,0.016129,0.609677,0.593548,0.016129,0,1.630645,1.535484,1.493548,0.474194,1.470968,2.056452,24.770968,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,12,11119,33,170,72,41,2,3542,28,24,6.0,0.0,4.346591,0.339015,26,82.224473,0.443182,0.231061,0.678030,0.000000,0.015152,0.585227,0.000000,0,2.410985,6.320076,2.248106,0.259470,2.369318,1.857955,53.361742,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,12,50835,28,175,68,22,1,1907,16,10,5.0,1.0,4.080952,0.176190,10,77.566742,0.900000,0.276190,1.109524,0.000000,0.000000,1.028571,0.000000,0,2.442857,6.071429,4.280952,0.247619,3.104762,4.576190,52.485714,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,12,83532,27,188,89,34,0,3040,24,2,5.0,0.0,3.696552,2.075862,6,71.118574,0.610345,0.103448,1.037931,0.000000,0.917241,0.424138,0.120690,0,0.865517,1.282759,1.458621,0.644828,0.951724,1.872414,19.334483,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,12,31772,30,183,76,41,4,3196,5,6,3.0,0.0,1.980952,0.254762,3,93.646575,2.050000,1.078571,0.816667,0.014286,0.352381,1.742857,0.000000,0,2.871429,1.011905,1.364286,0.111905,0.602381,0.895238,75.038095,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


## Sampling Data

Since the dataset is skewed, we must try different approaches

In [12]:
# Uniform Sampling Across Values
SEED = 420

gold_sample = rank_db[rank_db['fifa_overall_category'] == 1]
silver_sample = rank_db[rank_db['fifa_overall_category'] == 2]
bronze_sample = rank_db[rank_db['fifa_overall_category'] == 0]

sample_db = pd.concat([gold_sample, silver_sample, bronze_sample, bronze_sample])

card_ranks = rank_db.pop('fifa_overall_category')
s_card_ranks = sample_db.pop('fifa_overall_category')

# Build Dataset Normally
dataset = tf.data.Dataset.from_tensor_slices((rank_db.values, card_ranks.values))

sample_dataset =  tf.data.Dataset.from_tensor_slices((sample_db.values, s_card_ranks.values))

train_dataset = sample_dataset.shuffle(len(rank_db)).batch(1)


## Model Training & Evaluation

##### Model Creation Function

In [13]:

def get_compiled_model(opt='adam', act='relu', l='sparse_categorical_crossentropy' ):
  model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(3, activation="softmax"),
  ])

  model.compile(optimizer=opt,
                loss=l,
                metrics=['accuracy'])
  return model


### Model Tests

In [14]:

model = get_compiled_model()
model.fit(train_dataset, epochs=20, verbose=1)

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

6745/6745 [==============================] - 9s 1ms/step - loss: 112.2582 - accuracy: 0.4592
Epoch 2/20
6745/6745 [==============================] - 8s 1ms/step - loss: 1.9821 - accuracy: 0.4728
Epoch 3/20
6745/6745 [==============================] - 8s 1ms/step - loss: 1.2330 - accuracy: 0.4692
Epoch 4/20
6745/6745 [==============================] - 8s 1ms/step - loss: 1.1684 - accuracy: 0.4694
Epoch 5/20
6745/6745 [==============================] - 8s 1ms/step - loss: 1.4900 - accuracy: 0.4

KeyboardInterrupt: ignored

In [ ]:
train_predictions_baseline = model.predict(train_dataset, batch_size=BATCH_SIZE)

def plot_roc(name, labels, predictions, **kwargs):
  fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives [%]')
  plt.ylabel('True positives [%]')
  plt.xlim([-0.5,20])
  plt.ylim([80,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

plot_roc("Bruh", train_labels, train_predictions_baseline)

# Scaled NN Classifying Bronze and Not Bronze

### Data Initialization

In [22]:
# Feature Set for Predicting Card Level / Color
card_level_db = db[db.min_played >= 90]
card_level_db = pd.concat([card_level_db.iloc[:, 7:40].drop(columns=['ws_season']), card_level_db.iloc[:, -1]], axis=1)

#rank_db = card_level_db.drop(columns=['fifa_overall_category'])
rank_db = pd.get_dummies(card_level_db, drop_first=True, columns=['ws_tournament_name', 'position'])
rank_db.drop(['appearances'], axis=1)
display(rank_db.head())

AttributeError: ignored

In [16]:
# Change card rankings to numerical values (1,2,3)
rank_db['fifa_overall_category'] = pd.Categorical(rank_db['fifa_overall_category'])
rank_db['fifa_overall_category'] = rank_db.fifa_overall_category.cat.codes

# Make Bronze The Highlight
rank_db.loc[(rank_db.fifa_overall_category == 0 ),'fifa_overall_category'] = 3
rank_db.loc[(rank_db.fifa_overall_category == 1 ),'fifa_overall_category'] = 0
rank_db.loc[(rank_db.fifa_overall_category == 2 ),'fifa_overall_category'] = 0
rank_db.loc[(rank_db.fifa_overall_category == 3 ),'fifa_overall_category'] = 1

# Make Gold The Highlight
# rank_db.loc[(rank_db.fifa_overall_category == 2 ),'fifa_overall_category'] = 0


rank_db.head()

KeyError: ignored

### Preprocessing

In [ ]:
# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(rank_db, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('fifa_overall_category'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('fifa_overall_category'))
test_labels = np.array(test_df.pop('fifa_overall_category'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [ ]:
scaler = preprocessing.Normalizer()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

### Model

In [ ]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn')  
]

def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(64, activation='swish',input_shape=(train_features.shape[-1],)),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(lr=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

In [ ]:
EPOCHS = 100

model = make_model()
model.summary()


In [ ]:
model.fit(
    train_features,
    train_labels,
    epochs=50,
    validation_data=(val_features, val_labels), 
    verbose=1)

In [ ]:
def plot_cm(labels, predictions, p=0.5, n=None):
  cm = confusion_matrix(labels, predictions > p, normalize=n)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="f")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('(True Negatives): ', cm[0][0])
  print('(False Positives): ', cm[0][1])
  print('(False Negatives): ', cm[1][0])
  print('(True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))
baseline_results = model.evaluate(test_features, test_labels,
                                  batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, baseline_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_baseline)
plot_cm(test_labels, test_predictions_baseline, n='all')

### Oversampling

In [ ]:
pos_features = train_features[bool_train_labels]
neg_features = train_features[~bool_train_labels]

pos_labels = train_labels[bool_train_labels]
neg_labels = train_labels[~bool_train_labels]

ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))

res_pos_features = pos_features[choices]
res_pos_labels = pos_labels[choices]

res_pos_features.shape

In [ ]:
resampled_features = np.concatenate([res_pos_features, neg_features], axis=0)
resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]

resampled_features.shape

In [ ]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
  ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

pos_ds = make_ds(pos_features, pos_labels)
neg_ds = make_ds(neg_features, neg_labels)

for features, label in pos_ds.take(1):
  print("Features:\n", features.numpy())
  print()
  print("Label: ", label.numpy())

In [ ]:
resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(BATCH_SIZE).prefetch(2)

for features, label in resampled_ds.take(1):
  print(label.numpy().mean())

In [ ]:
resampled_steps_per_epoch = np.ceil(2.0*neg/BATCH_SIZE)
resampled_steps_per_epoch

In [ ]:
resampled_model = make_model()

# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).cache()
val_ds = val_ds.batch(BATCH_SIZE).prefetch(2) 

resampled_history = resampled_model.fit(
    resampled_ds,
    epochs=EPOCHS,
    steps_per_epoch=resampled_steps_per_epoch,
    callbacks=[early_stopping],
    validation_data=val_ds)

In [ ]:
train_predictions_resampled = resampled_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_resampled = resampled_model.predict(test_features, batch_size=BATCH_SIZE)

resampled_results = resampled_model.evaluate(test_features, test_labels,
                                             batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(resampled_model.metrics_names, resampled_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_resampled)


In [ ]:
plot_cm(test_labels, test_predictions_resampled, n='pred')

### Results


#### Model

In [ ]:
plot_cm(test_labels, test_predictions_baseline)


In [ ]:
plot_cm(test_labels, test_predictions_baseline, n='all')

#### Oversampling

In [ ]:

plot_cm(test_labels, test_predictions_resampled)

In [ ]:
plot_cm(test_labels, test_predictions_resampled, n='all')

#### ROCs

In [ ]:
def plot_roc(name, labels, predictions, **kwargs):
  fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives [%]')
  plt.ylabel('True positives [%]')
  # plt.xlim([-0.5,20])
  # plt.ylim([80,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plt.legend(loc='lower right')